# Enigma and Bombe Simulator Documentation

### Introduction
This documentation provides an in-depth look at the EnigmaSimulator and BombeSimulator classes, simulating the operations of the historic Enigma machine and Bombe device. The Enigma machine, used during World War II, is an encryption device known for its complexity and the Bombe was a device designed to decrypt Enigma-encoded messages.

### EnigmaSimulator Class
##### Overview
The EnigmaSimulator class is a Python implementation that simulates the workings of the Enigma machine. It includes functionalities to set up rotors, reflectors, and plugboards, and to encrypt or decrypt messages.

##### Components
**Alphabet**
- The machine uses a standard Latin alphabet ('ABCDEFGHIJKLMNOPQRSTUVWXYZ') for encryption and decryption processes.

**Rotor Collection**
- A dictionary mapping each rotor type (I through VIII) to its corresponding encryption mapping.
- Each rotor has a unique character substitution pattern.

**Notches**
- Each rotor type has specified notch positions that trigger the next rotor's rotation, mimicking the mechanical movement of the original Enigma machine.

**Reflector Collection**
- Contains different reflector types (A, B, C, Bt, Ct), each with a unique substitution pattern. The reflector is a critical component that adds to the complexity of the encryption.

##### Methods
**set_rotors**
- Configures the rotors with given names and initial positions.

**set_reflector**
- Sets the reflector based on the given name.

**configure_plugboard**
- Configures the plugboard with given character mappings, adding an additional layer of complexity to the encryption.

**rotate_rotors**
- Simulates the rotation of the rotors, a key aspect of the Enigma's encryption mechanism.

**encrypt_decrypt_char**
- Encrypts or decrypts a single character, encapsulating the core functionality of the Enigma machine.

**encrypt_decrypt**
- Encrypts or decrypts an entire message character by character.

**reset_rotors**
- Resets the rotors to their initial positions, an essential step for accurate decryption.

In [2]:
class EnigmaSimulator:
    def __init__(self):
        # The alphabet used for encryption and decryption
        self.alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

        # Collection of rotors with their respective mappings
        # Each key represents a rotor type, with its value being the encryption mapping
        self.rotorCollection = {
            'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ',
            'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE',
            'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO',
            'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB',
            'V': 'VZBRGITYUPSDNHLXAWMJQOFECK',
            'VI': 'JPGVOUMFYQBENHZRDKASXLICTW',
            'VII': 'NZJHGRCXMYSWBOUFAIVLPEKQDT',
            'VIII': 'FKQHTLXOCBJSPDZRAMEWNIUYGV'
            }
        
        # Notches that determine when the next rotor should rotate
        # Each key is a rotor type, with its value indicating the notch positions
        self.notches = {
            'I': 'Q',
            'II': 'E',
            'III': 'V',
            'IV':'J',
            'V': 'Z',
            'VI': 'ZM',
            'VII': 'ZM',
            'VIII': 'ZM'
            }
        
        # Collection of reflectors with their mappings
        # Each key is a reflector type, with its value being the reflection mapping
        self.reflectorCollection = {
            'A': 'EJMZALYXVBWFCRQUONTSPIKHGD',
            'B': 'YRUHQSLDPXNGOKMIEBFZCWVJAT',
            'C': 'FVPJIAOYEDRZXWGCTKUQSBNMHL',
            'Bt': 'ENKQAUYWJICOPBLMDXZVFTHRGS',
            'Ct': 'RDOBJNTKVEHMLFCWZAXGYIPSUQ'
            }
        

        self.rotors = [] # Stores the selected rotors and their positions
        self.reflector = None # Stores the selected reflector
        self.plugboard = {} # Configuration of the plugboard

    def set_rotors(self, rotor_names, positions):
        """Configures the rotors with given names and initial positions"""
        self.rotors = [[list(self.rotorCollection[name]), self.alphabet.index(position)] for name, position in zip(rotor_names, positions)]

    def set_reflector(self, reflector_name):
        """Sets the reflector based on the given name"""
        self.reflector = self.reflectorCollection[reflector_name]

    def configure_plugboard(self, mapping):
        """Configures the plugboard with given character mappings"""
        self.plugboard = {k: v for k, v in mapping.items()}
        self.plugboard.update({v: k for k, v in mapping.items()})

    def rotate_rotors(self):
        """Rotates the rotors, with cascading rotation based on the notch positions"""
        rotate_next = True
        for i in range(len(self.rotors)):
            rotor, position = self.rotors[i]
            if rotate_next or self.alphabet[position] in self.notches[list(self.rotorCollection.keys())[i]]:
                self.rotors[i][1] = (position + 1) % len(self.alphabet)
                rotate_next = self.alphabet[position] in self.notches[list(self.rotorCollection.keys())[i]]
            else:
                break

    def encrypt_decrypt_char(self, char):
        """Encrypts or decrypts a single character through the Enigma machine process"""
        if char in self.plugboard:
            char = self.plugboard[char]

        # Rotor encryption with position adjustment
        for rotor, position in self.rotors[::-1]:
            char_idx = (self.alphabet.index(char) + position) % len(self.alphabet)
            char = rotor[char_idx]
            char = self.alphabet[(self.alphabet.index(char) - position) % len(self.alphabet)]

        # Reflector
        char = self.reflector[self.alphabet.index(char)]

        # Rotor decryption with position adjustment
        for rotor, position in self.rotors:
            char_idx = (self.alphabet.index(char) + position) % len(self.alphabet)
            char = self.alphabet[rotor.index(self.alphabet[char_idx])]
            char = self.alphabet[(self.alphabet.index(char) - position) % len(self.alphabet)]

        if char in self.plugboard:
            char = self.plugboard[char]

        return char

    def encrypt_decrypt(self, message):
        """Encrypts or decrypts an entire message character by character"""
        encrypted_message = ''
        for char in message.upper():
            self.rotate_rotors()
            if char in self.alphabet:
                encrypted_message += self.encrypt_decrypt_char(char)
            else:
                encrypted_message += char
        return encrypted_message
    
    def reset_rotors(self):
        """Resets the rotation count of each rotor to its initial position"""
        for rotor in self.rotors:
            rotor[1] = 0

In [3]:
enigma = EnigmaSimulator()
enigma.set_rotors(['I', 'IV', 'II'], ['C', 'B', 'A'])
enigma.set_reflector('B')
enigma.configure_plugboard({'A': 'B', 'C': 'D', 'V': 'J', 'F': 'E'})

message = "Hello world!"
encrypted = enigma.encrypt_decrypt(message)
print(f"Encrypted: {encrypted}")

# Reset the Enigma machine to the initial settings for decryption
enigma.reset_rotors()  # Reset the rotor positions
enigma.set_rotors(['I', 'IV', 'II'], ['C', 'B', 'A'])
decrypted = enigma.encrypt_decrypt(encrypted)
print(f"Decrypted: {decrypted}")

Encrypted: ZLKSU YCGXJ!
Decrypted: HELLO WORLD!


In [4]:
class BombeSimulator:
    def __init__(self):
        """Initialize with possible Enigma settings"""
        pass

    def decrypt_message(self, encrypted_message):
        """Decrypt an encrypted message by simulating the Bombe's process"""
        pass